In [7]:
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go

In [8]:
# Read CSV file

expenses_df = pd.read_csv('folder/expenses.csv')
expenses_df['date'] = pd.to_datetime(expenses_df['date'])

print(expenses_df.head())
print('\n\n')
print(expenses_df.dtypes)

        date  category            type    cost
0 2020-04-29  shopping         redtick   25.40
1 2020-05-01   eat out        Dominoes   40.06
2 2020-05-02  shopping  village grocer   72.40
3 2020-05-09  shopping  village grocer  121.75
4 2020-05-09   eat out  MY81 Hyderabad   40.00



date        datetime64[ns]
category            object
type                object
cost               float64
dtype: object


In [9]:
# Find the missing dates

# Find the earliest and latest dates
def findEarliestLatestDates(df):
    earliest_date = df['date'].iloc[0]
    
    return earliest_date.date(), datetime.now().date()

# Missing dates = no expenses covered
def findRangeOfZeroExpenseDates(earliest_date, latest_date):
    
    date_range = pd.date_range(start = earliest_date, end = latest_date)
    zero_expense_dates = date_range.difference(expenses_df['date'])
    
    return zero_expense_dates

def createZeroExpenseInfo(date):
    return {
        'date': date,
        'category': 'zero expenses',
        'type': '-',
        'cost': 0.00
    }

# Append to dataframe
def appendMissingDates(expenses_df, zero_expense_dates):
    
    zero_expense_dict_list = [
        createZeroExpenseInfo(date) for date in zero_expense_dates
    ]
        
    expenses_df = expenses_df.append(zero_expense_dict_list, ignore_index=True)
    
    return expenses_df.sort_values(by='date',ascending=True)

# Get the earliest and latest date
earliest_date, latest_date = findEarliestLatestDates(expenses_df)

zero_expense_dates = findRangeOfZeroExpenseDates(earliest_date, latest_date)

expenses_df = appendMissingDates(expenses_df, zero_expense_dates)

print(expenses_df)

         date       category            type   cost
0  2020-04-29       shopping         redtick  25.40
89 2020-04-30  zero expenses               -   0.00
1  2020-05-01        eat out        Dominoes  40.06
2  2020-05-02       shopping  village grocer  72.40
90 2020-05-03  zero expenses               -   0.00
..        ...            ...             ...    ...
85 2020-07-14    maintenance             car  31.20
84 2020-07-14        eat out     family mart  10.10
86 2020-07-15       shopping     jaya grocer  11.30
87 2020-07-16        eat out   secret recipe  10.00
88 2020-07-17        eat out      food truck  12.00

[116 rows x 4 columns]


In [10]:
# Time series plot
costs_df = expenses_df[['date','cost']]
costs_df = costs_df.groupby(['date']).sum().reset_index()

# Begin plotting here
fig = go.Figure([
    go.Scatter(
        x=costs_df['date'], 
        y=costs_df['cost'],
        mode='lines+markers'
    )
])

fig.update_layout(title_text='Overview of expenses')
fig.update_xaxes(rangeslider_visible=True)
fig.show()


In [28]:
category_counts_ser = expenses_df['category'].value_counts()

categories = category_counts_ser.keys()
values = category_counts_ser.values

fig = go.Figure([go.Bar(x=categories, y=values)])

fig.update_layout(
    title='Count categories of expenses',
    yaxis=dict(
        title='Counts'
    ),
    xaxis=dict(
        title='Categories'
    )
)

fig.show()

In [32]:
expenses_only_df = expenses_df[expenses_df['category'] != 'zero expenses']
non_expenses_df = expenses_df[expenses_df['category'] == 'zero expenses']

fig = go.Figure([go.Bar(x=['Expenses','Zero expenses'], y=[len(expenses_only_df),len(non_expenses_df)])])

fig.update_layout(
    title='Counts: Expenses vs Zero Expenses',
    yaxis=dict(
        title='Counts'
    ),
    xaxis=dict(
        title='Categories'
    ),
    width=600,
    height=600
)

fig.show()